# Separations Processes: Designing a Distillation Column with McCabe-Thiele

**Objective:** This lesson provides a hands-on guide to designing a binary distillation column using the graphical McCabe-Thiele method. We will translate the graphical method into a computational algorithm to determine the number of theoretical stages required for a specific separation.

**Learning Goals:**
1.  Understand the core concepts of Vapor-Liquid Equilibrium (VLE) for binary systems.
2.  Derive and understand the meaning of the Operating Lines (Rectifying and Stripping) and the Feed Line (q-line).
3.  Construct a complete McCabe-Thiele diagram programmatically.
4.  Implement an algorithm to "step off" the theoretical stages and find the optimal feed location.
5.  Analyze how key design parameters, like the reflux ratio and feed condition, impact the column design.

## The Problem: Binary Distillation

Distillation is the most widely used separation technique in the chemical industry. It works by exploiting differences in **volatility** between components in a liquid mixture. We will design a column to separate a mixture of a more volatile component (MVC) and a less volatile component (LVC).

![Distillation Column Diagram](https://www.shutterstock.com/image-vector/vector-illustration-distillation-column-eps10-600nw-2207077179.jpg)

**Our Goal:** Given a feed stream with a certain composition ($z_F$) and thermal condition ($q$), we want to produce a high-purity **distillate** ($x_D$) rich in the MVC and a high-purity **bottoms** ($x_B$) rich in the LVC. Our task is to calculate the **number of theoretical stages** required to achieve this separation at a given **reflux ratio ($R$)**.

## Part 1: The Physics - Vapor-Liquid Equilibrium (VLE)

The foundation of the McCabe-Thiele method is the **equilibrium curve**, which plots the vapor mole fraction ($y$) versus the liquid mole fraction ($x$) for the more volatile component. For many ideal systems, this relationship can be described using a **constant relative volatility ($\alpha$)**.
$$ y = \frac{\alpha x}{1 + (\alpha - 1)x} $$
A higher $\alpha$ means the separation is easier. The equilibrium curve always lies above the $y=x$ line (also called the 45-degree line).

## Part 2: The Model - Operating & Feed Lines

While the equilibrium curve describes the thermodynamics on each stage, **operating lines** describe the bulk movement of liquid and vapor between stages, based on mass balances.

**1. Rectifying Section (Top) Operating Line (ROL):** This relates the compositions of vapor ($y_{n+1}$) rising from a stage to the liquid ($x_n$) entering it in the section above the feed. Its equation depends on the **reflux ratio ($R$)** and the **distillate composition ($x_D$)**.
$$ y_{n+1} = \frac{R}{R+1}x_n + \frac{1}{R+1}x_D $$

**2. Stripping Section (Bottom) Operating Line (SOL):** This describes the section below the feed. It relates to the **bottoms composition ($x_B$)**.

**3. Feed Line (q-line):** This line represents the locus of all possible intersection points of the two operating lines. Its slope is determined by the thermal condition of the feed, $q$.
$$ y = \frac{q}{q-1}x - \frac{z_F}{q-1} $$
*   $q=1$: Saturated Liquid (vertical line)
*   $q=0$: Saturated Vapor (horizontal line)
*   $0 < q < 1$: Two-phase mixture
*   $q > 1$: Subcooled liquid
*   $q < 0$: Superheated vapor

## Part 3: Python Implementation and Diagram Construction

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt

# --- Define System Parameters ---
# VLE Data
alpha = 2.5       # Relative volatility of the MVC

# Feed and Product Specifications
zF = 0.50         # Mole fraction of MVC in feed
xD = 0.95         # Desired mole fraction of MVC in distillate
xB = 0.05         # Desired mole fraction of MVC in bottoms

# Operating Conditions
R = 2.0           # Reflux Ratio (L/D)
q = 1.0           # Feed quality (q=1 means saturated liquid feed)

print("System parameters are defined.")

In [ ]:
# --- Define Functions for Each Line ---

def equilibrium_curve(x, alpha):
    """Calculates y from x based on relative volatility."""
    return alpha * x / (1 + (alpha - 1) * x)

def rol(x, R, xD):
    """Rectifying Operating Line equation."""
    return (R / (R + 1)) * x + (xD / (R + 1))

def q_line(x, zF, q):
    """q-line equation."""
    # Handle the vertical line case for q=1
    if abs(q - 1.0) < 1e-6:
        return np.linspace(0, 1, 100) # Placeholder, handled in plotting
    return (q / (q - 1)) * x - (zF / (q - 1))

# --- Calculate Operating Line Intersections ---
# Find the intersection of the ROL and the q-line
# This point defines where the two operating lines meet
# We solve: rol(x) = q_line(x) for x
A = (R / (R + 1)) - (q / (q - 1))
B = (zF / (q - 1)) + (xD / (R + 1))
x_intersect = B / A
y_intersect = rol(x_intersect, R, xD)

# The SOL connects (xB, xB) to the intersection point
m_sol = (y_intersect - xB) / (x_intersect - xB)
def sol(x, m_sol, xB):
    """Stripping Operating Line equation."""
    return m_sol * (x - xB) + xB

print("Line functions and intersection points calculated.")

## Part 4: Stepping Off the Stages

Now for the exciting part. We will start at the top of the column (at $x=x_D$) and computationally trace the staircase down to the bottom ($x_B$), counting the steps. Each step represents one theoretical equilibrium stage.

In [ ]:
# --- Stage Calculation Algorithm ---

x_steps = [xD]  # Start with the distillate composition
y_steps = [xD]

num_stages = 0
feed_stage = 0

x_current = xD
y_current = xD

while x_current > xB:
    num_stages += 1
    
    # Horizontal step: from operating line to equilibrium curve
    # We need to find the x that gives y_current on the eq. curve
    # y = alpha*x / (1 + (alpha-1)x) -> x = y / (alpha - y*(alpha-1))
    x_next = y_current / (alpha - y_current * (alpha - 1))
    x_steps.extend([x_next, x_next])
    y_steps.extend([y_current, y_current])

    # Vertical step: from equilibrium curve to the next operating line
    # Check if we have crossed the feed stage intersection point
    if x_next > x_intersect:
        y_next = rol(x_next, R, xD)
    else:
        # Record the feed stage the first time we switch operating lines
        if feed_stage == 0:
            feed_stage = num_stages
        y_next = sol(x_next, m_sol, xB)
        
    x_steps.append(x_next)
    y_steps.append(y_next)
    
    # Update current position for the next loop
    x_current = x_next
    y_current = y_next

print(f"Calculation Complete!")
print(f"  - Total Theoretical Stages (including reboiler): {num_stages}")
print(f"  - Optimal Feed Stage: {feed_stage}")

## Part 5: The Final Diagram and Analysis

Let's visualize the entire construction.

In [ ]:
# --- Plotting the McCabe-Thiele Diagram ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(10, 10))

# Generate x-values for smooth curves
x_range = np.linspace(0, 1, 100)

# Plot the equilibrium curve and y=x line
plt.plot(x_range, equilibrium_curve(x_range, alpha), label='Equilibrium Curve')
plt.plot(x_range, x_range, 'k-', label='y=x')

# Plot the operating lines
plt.plot([xB, x_intersect], [xB, y_intersect], 'g-', label='Stripping Op. Line')
plt.plot([x_intersect, xD], [y_intersect, xD], 'r-', label='Rectifying Op. Line')

# Plot the q-line
if abs(q - 1.0) < 1e-6:
    plt.plot([zF, zF], [zF, rol(zF, R, xD)], 'm--', label='q-line (Sat. Liquid)')
else:
    plt.plot([zF, x_intersect], [zF, y_intersect], 'm--', label='q-line')

# Plot the stepped-off stages
plt.plot(x_steps, y_steps, 'b-', label=f'{num_stages} Stages')

# Formatting
plt.title('McCabe-Thiele Diagram for Binary Distillation', fontsize=16, weight='bold')
plt.xlabel('Liquid Mole Fraction, x (MVC)', fontsize=12)
plt.ylabel('Vapor Mole Fraction, y (MVC)', fontsize=12)
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.legend()
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

## Part 6: You are the Separations Engineer!

The most important part of process design is understanding trade-offs. Use this simulation to explore them. Modify the **Operating Conditions** in **Part 3** and re-run all the cells.

#### Challenge 1: The Reflux Ratio Trade-off
Increase the **Reflux Ratio ($R$)** from `2.0` to `4.0`. 
*Before you run, predict*: What will happen to the Rectifying Operating Line? Will you need more or fewer stages to achieve the same separation? What is the downside of a higher reflux ratio in a real plant? (Hint: think about the energy required to boil the liquid for the reflux).

#### Challenge 2: The Minimum Number of Stages
What is the absolute minimum number of stages required for this separation? This occurs at **total reflux**, where $R$ is infinite. You can simulate this by setting $R$ to a very large number, like `10000`. What happens to the operating lines in this case?

#### Challenge 3: Effect of Feed Quality
Reset $R$ to 2.0. Now, change the feed from a saturated liquid ($q=1.0$) to a **50/50 vapor-liquid mixture ($q=0.5$)**. 
*Predict*: How will the slope and position of the q-line change? Will this change the number of stages required?